In [ ]:
from pyannote.audio import Inference
import numpy as np
from pyannote.core import Segment, Timeline

In [ ]:
model = Inference("hbredin/SpeakerEmbedding-XVectorMFCC-VoxCeleb", window="sliding", duration=1, step=1)

In [ ]:
embeddings = model("/Users/andrei-macpro/Documents/Data/Audio/no_os2.0_1043.wav")

In [ ]:
len(embeddings)

In [ ]:
for embedding in embeddings:
    print(embedding)

In [ ]:
for embedding in embeddings:   # for each 1 seconds Segment there's an embedding (an array)
    for segment in embedding:
        print(segment)

In [ ]:
for embedding in embeddings:   # for each 1 seconds segment there's an embedding
    print(type(embedding))

In [ ]:
so the embeddings contain a segment:array tuple

In [ ]:
dict_embeddings=dict(embeddings)

In [ ]:
(dict(embeddings)).values()

In [ ]:
dict_embeddings.values()

In [ ]:
dict_embeddings.keys()

In [ ]:
for array in dict_embeddings.values():
    print(len(array))

In [ ]:
no_512_embeddings = [len(array) for array in dict_embeddings.values()]

In [ ]:
len(no_512_embeddings) #!! ok so each second we have a 512 embedding
#ok so for clustering i need each row to be an embedding
# and then i can get the size of the clusters but how i go from embedding back to timestamps - i can use the tuple?
# in order to do re-segmentation i have to associate each segment's label back to annotation and then add those to timeline


In [ ]:
dict_embeddings.keys()

In [ ]:
array_embeddings = np.array(list(dict_embeddings.values()))

In [ ]:
array_embeddings.shape #nicee i can now use this for clustering 

In [ ]:
len(dict_embeddings.keys())

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=3)

In [ ]:
kmeans

In [ ]:
kmeans.labels_

In [ ]:
predictions = kmeans.fit_predict(array_embeddings)

In [ ]:
segments0 
for key, value in zip(dict_embeddings.keys(),predictions):
    print(key, value)

In [ ]:
cluster0 = np.array([embedding for embedding, cluster in zip(array_embeddings, predictions) if cluster==0])

In [ ]:
cluster0_pred = np.array([label for label in predictions if label==0])

In [ ]:
cluster1= np.array([embedding for embedding, cluster in zip(array_embeddings, predictions) if cluster==1])

In [ ]:
cluster1_pred = np.array([label for label in predictions if label==1])

In [ ]:
new_cluster = np.concatenate((cluster0, cluster1), axis=0)

In [ ]:
new_predictions = np.concatenate((cluster0_pred, cluster1_pred), axis=0)

In [ ]:
predictions, new_predictions

In [ ]:
segments0 = [key for key, value in zip(dict_embeddings.keys(),predictions) if value==0]
timeline0 = Timeline(segments=segments0, uri='/Users/andrei-macpro/Documents/Data/Audio/no_os2.0_1043.wav')

In [ ]:
segments0[0].start

In [ ]:
print(timeline0[0])

In [ ]:
timeline1 = Timeline(segments=[key for key, value in zip(dict_embeddings.keys(),predictions) if value==1],
                     uri='/Users/andrei-macpro/Documents/Data/Audio/no_os2.0_1043.wav')

In [ ]:
timeline2 = Timeline(segments=[key for key, value in zip(dict_embeddings.keys(),predictions) if value==2],
                     uri='/Users/andrei-macpro/Documents/Data/Audio/no_os2.0_1043.wav')

In [ ]:
timeline3 = Timeline(segments=[key for key, value in zip(dict_embeddings.keys(),predictions) if value==3],
                     uri='/Users/andrei-macpro/Documents/Data/Audio/no_os2.0_1043.wav')

In [ ]:
for segment in timeline2:  # mostly cg
    play(file_pydub[segment.start*1000:segment.end*1000])

In [ ]:
cluster 3 = cg
cluster 2 = bb


In [ ]:
len(timeline0) , len(timeline1), len(timeline2)

In [ ]:
bb_speech=[file_pydub[segment.start*1000:segment.end*1000] for segment in timeline0]

In [ ]:
sum(bb_speech).export('/Users/andrei-macpro/Documents/Data/Audio/bb_speech.wav', format="wav")

In [ ]:
for segment in timeline1: # mostly cg
    play(file_pydub[segment.start*1000:segment.end*1000])

In [ ]:
cg_speech=[file_pydub[segment.start*1000:segment.end*1000] for segment in timeline1]

sum(cg_speech).export('/Users/andrei-macpro/Documents/Data/Audio/cg_speech.wav', format="wav")

In [ ]:
for segment in timeline2: # mostly background noises (or sounds that don't sound like speech + experimenter)
    play(file_pydub[segment.start*1000:segment.end*1000])

In [ ]:
for segment in timeline3: # pauses in speech 
    play(file_pydub[segment.start*1000:segment.end*1000])

# clusters for window of 1

In [ ]:
# let's vizualize the clusters   
plt.figure(figsize=(10,10))
scatter = plt.scatter(array_embeddings[:, 0], array_embeddings[:, 1], c=predictions, s=50, cmap='viridis')
plt.legend(*scatter.legend_elements())
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);


# clusters with window of 2

In [ ]:
plt.figure(figsize=(10,10))
scatter = plt.scatter(array_embeddings[:, 0], array_embeddings[:, 1], c=predictions, s=50, cmap='viridis')
plt.legend(*scatter.legend_elements())
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

In [ ]:
plt.figure(figsize=(10,10))
scatter = plt.scatter(new_cluster[:, 0], new_cluster[:, 1], c=new_predictions)

In [ ]:
cluster0[0]

In [ ]:
# what if i only plot 0 and 1? 


In [ ]:
scatter.legend_elements()

In [ ]:
predictions

In [ ]:
# omg maybe the input should be just the voiced parts....... 

In [ ]:
sound0=Timeline()
sound1=Timeline()
sound2=Timeline()
for key, value in zip(dict_embeddings.keys(),predictions):
    if value==0:
        sound0.add(key)
    elif value==1:
        sound1.add(key)
    else:
        sound2.add(key)

In [ ]:
print(sound1)

In [ ]:
from pydub import AudioSegment
file_pydub = AudioSegment.from_wav("/Users/andrei-macpro/Documents/Data/Audio/no_os2.0_1043.wav")

In [ ]:
timestamps_0=dict(sound0)
start_times_sound0=[x*1000 for x in list(timestamps_0.keys())]
end_times_sound0=[x*1000 for x in list(timestamps_0.values())]

In [ ]:
timestamps_1=dict(sound1)
start_times_sound1=[x*1000 for x in list(timestamps_1.keys())]
end_times_sound1=[x*1000 for x in list(timestamps_1.values())]

In [ ]:
timestamps_2=dict(sound2)
start_times_sound2=[x*1000 for x in list(timestamps_2.keys())]
end_times_sound2=[x*1000 for x in list(timestamps_2.values())]

In [ ]:
cluster0=[file_pydub[start_time:end_time] for start_time,end_time in zip(start_times_sound0, end_times_sound0)]

In [ ]:
cluster1=[file_pydub[start_time:end_time] for start_time,end_time in zip(start_times_sound1, end_times_sound1)]

In [ ]:
cluster2=[file_pydub[start_time:end_time] for start_time,end_time in zip(start_times_sound2, end_times_sound2)]

In [ ]:
sum(speech_segments).export('/Users/andrei-macpro/Documents/Data/Audio/1043_bb', format="wav")

In [ ]:

for sound in cluster0:
    play(sound)

In [ ]:
from pydub.playback import play

In [ ]:
from pydub.playback import play
for sound in cluster1:
    play(sound)

In [ ]:
for sound in cluster2:
    play(sound)

In [ ]:
import matplotlib.pyplot as plt
sse = []
list_k = list(range(1, 6))

for k in list_k:
    km = KMeans(n_clusters=k)
    km.fit(array_embeddings)
    sse.append(km.inertia_)

# Plot sse against k
plt.figure(figsize=(6, 6))
plt.plot(list_k, sse, '-o')
plt.xlabel(r'Number of clusters *k*')
plt.ylabel('Sum of squared distance');

In [ ]:
km